<a href="https://colab.research.google.com/github/Dforouzanfar/ML_Unimi_2024/blob/master/muffin_chihuahua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import zipfile
from pathlib import Path

In [ ]:
# If you want to download the dataset, you should insert your own credentials.

os.environ['KAGGLE_USERNAME'] = "**"
os.environ['KAGGLE_KEY'] = "**"

In [ ]:
# Setup path to a data folder
data_path = Path("data/")
image_path = data_path / "muffin_chihuahua"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  pass
else:
  image_path.mkdir(parents=True, exist_ok=True)

# Download dataset
os.chdir(data_path)
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

os.chdir("/content/")
with zipfile.ZipFile("/content/data/muffin-vs-chihuahua-image-classification.zip", "r") as zip_ref:
  print("Unzipping")
  zip_ref.extractall(image_path)

Dataset URL: https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification
License(s): CC0-1.0
 98% 464M/474M [00:02<00:00, 198MB/s]
100% 474M/474M [00:02<00:00, 204MB/s]
Unzipping
